<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
import snowflake.connector
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
load_dotenv()

# Retrieve Snowflake credentials from environment variables
user = os.getenv('SNOWFLAKE_USER')
password = os.getenv('SNOWFLAKE_PASSWORD')
account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = 'SF_WH_CASE1'
database = 'SF_DB_CASE1'
schema = 'SF_CASE1'
role = 'SYSADMIN'  # Replace with your full access role

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role=role,
)

# Create a cursor object
cur = conn.cursor()

try:
#     cur.execute(f"CREATE ROLE IF NOT EXISTS {role}")
#     cur.execute(f"GRANT ALL PRIVILEGES ON DATABASE {database} TO ROLE {role}")
#     cur.execute(f"GRANT ALL PRIVILEGES ON WAREHOUSE {warehouse} TO ROLE {role}")
#     cur.execute(f"GRANT ROLE {role} TO USER {user}")

    # Execute SQL commands for setup
    cur.execute("CREATE DATABASE IF NOT EXISTS SF_DB_CASE1")
    cur.execute("USE DATABASE SF_DB_CASE1")
    cur.execute("""
        CREATE WAREHOUSE IF NOT EXISTS SF_WH_CASE1
        WITH WAREHOUSE_SIZE = 'MEDIUM'
        WAREHOUSE_TYPE = 'STANDARD'
        AUTO_SUSPEND = 300
        AUTO_RESUME = TRUE;
    """)
    cur.execute("CREATE SCHEMA IF NOT EXISTS SF_CASE1")
    cur.execute("USE SCHEMA SF_CASE1")
    cur.execute("""
        CREATE OR REPLACE TABLE SUMMARY_FINAL (
            NAME_OF_THE_TOPIC VARCHAR(500),
            YEAR VARCHAR(255),
            LEVEL VARCHAR(255),
            INTRODUCTION_SUMMARY VARCHAR(10000),
            LEARNING_OUTCOMES VARCHAR(10000),
            LINK_TO_THE_SUMMARY_PAGE VARCHAR(1000),
            LINK_TO_THE_PDF_FILE VARCHAR(1000)
        );
    """)

    print("Snowflake setup completed successfully.")

    # Load CSV data into DataFrame
    csv_file_path = '../Webscrape/extracted.csv'  # Update with the path to your CSV file
    df = pd.read_csv(csv_file_path)

    # Prepare DataFrame for Snowflake
    df.columns = [col.upper() for col in df.columns]

    # Configure SQLAlchemy engine for Snowflake
    engine = create_engine(URL(
        account=account,
        user=user,
        password=password,
        database=database,
        schema=schema,
        warehouse=warehouse,
        role=role,
    ))

    # Transfer data from CSV to Snowflake
    table_name = 'SUMMARY_FINAL'
    df.to_sql(table_name, con=engine, index=False, if_exists='append', method='multi')

    print("Data transfer to Snowflake completed successfully.")
finally:
    # Clean up
    cur.close()
    conn.close()

ProgrammingError: 251005: User is empty